In [10]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
!pip install split-folders
import splitfolders

from tensorflow.keras.applications import EfficientNetB0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
output = "sortedData"
splitfolders.ratio('/content/drive/MyDrive/coffee/coffeedataset40', output=output, seed=42, ratio=(0.7, 0.15, 0.15))
train_dir = "sortedData/train"
test_dir = "sortedData/test"
val_dir = "sortedData/val"

Copying files: 80 files [00:15,  5.11 files/s]


In [14]:
epochs = 5
NUM_CLASSES = 2
model_save_location = "Model/EfficientNet"

In [15]:
img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def build_model(NUM_CLASSES):
    inputs = layers.Input(shape=(224, 224, 3))
    x = img_augmentation(inputs)

    #Using the imported version of EfficientNet
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [16]:
model = build_model(NUM_CLASSES)
unfreeze_model(model)

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=train_dir, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=val_dir, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=test_dir, target_size=(224,224), batch_size=10, shuffle=False)

_ = model.fit(train_batches, epochs=epochs, validation_data=valid_batches, verbose=1)

16705208/16705208 [==============================] - 0s 0us/step
Found 777 images belonging to 2 classes.
Found 171 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
Epoch 1/5
78/78 [==============================] - 58s 647ms/step - loss: 0.2588 - accuracy: 0.8932 - val_loss: 0.1497 - val_accuracy: 0.9766
Epoch 2/5
78/78 [==============================] - 52s 666ms/step - loss: 0.0908 - accuracy: 0.9665 - val_loss: 0.0396 - val_accuracy: 1.0000
Epoch 3/5
78/78 [==============================] - 51s 653ms/step - loss: 0.0980 - accuracy: 0.9601 - val_loss: 0.0180 - val_accuracy: 1.0000
Epoch 4/5
78/78 [==============================] - 52s 669ms/step - loss: 0.0470 - accuracy: 0.9794 - val_loss: 0.0064 - val_accuracy: 1.0000
Epoch 5/5
78/78 [==============================] - 48s 613ms/step - loss: 0.0711 - accuracy: 0.9717 - val_loss: 0.0114 - val_accuracy: 1.0000


In [17]:
#Testing the Model
test_labels = test_batches.classes
print("Test Labels",test_labels)
print(test_batches.class_indices)

predictions = model.predict(test_batches,steps=len(test_batches),verbose=0)

acc = 0
for i in range(len(test_labels)):
    actual_class = test_labels[i]
    if predictions[i][actual_class] > 0.5 : 
        acc += 1
print("Accuracy:",(acc/len(test_labels))*100,"%")

Test Labels [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
{'darkbean': 0, 'espresso': 1}
Accuracy: 100.0 %
